# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report




In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('Msg_Category', engine)
all_cols = df.columns.to_list()
x_cols = all_cols[0:3]
y_cols =  all_cols
del y_cols[0:3]

df_X = df[x_cols]
df_Y = df[y_cols]
df_Y = df_Y.drop(['child_alone'], axis=1)

X = df_X.message.values
y = df_Y.values

In [3]:
y.shape

(26180, 35)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = sklearn.pipeline.Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
('tfidf', TfidfTransformer()),
('clf', MultiOutputClassifier(LogisticRegression()))])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=43)

In [7]:
for i in range(0,35):
    if (sum(y_test[:,i]==1) == 0) | (sum(y_test[:,i]==1) == 0):
        print(i)

In [8]:
pipeline.fit(X_train, y_train)
# predict on test data
y_pred = pipeline.predict(X_test)

c:\Users\avadi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
labels = np.unique(y_pred)
#confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
accuracy = (y_pred == y_test).mean()

print("Labels:", labels)
#print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:", accuracy)

Labels: [0 1]
Accuracy: 0.948234126984127


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
Precision = []
Recall = []
F1 = []
Acc = []
for i in range(y_test.shape[1]):
    report = classification_report(y_test[:, i], y_pred[:, i], output_dict=True)
    
    macro_precision = report['macro avg']['precision']
    Precision.append(macro_precision)
    
    macro_recall = report['macro avg']['recall']
    Recall.append(macro_recall)
    
    macro_f1_score = report['macro avg']['f1-score']
    F1.append(macro_f1_score)

    accuracy = report['accuracy']
    Acc.append(accuracy)



c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metr

In [13]:
cats = df_Y.columns.to_list()


Evaluate_1 = pd.DataFrame(
    {'Category': cats,
     'Precision' : Precision,
     'Recall' : Recall,
     'Accuracy': Acc,
     'F1': F1
    })

Evaluate_1

,Category,Precision,Recall,Accuracy,F1
0,related,0.787441,0.681940,0.827083,0.709848
1,request,0.848753,0.754150,0.897338,0.789433
2,offer,0.498206,0.500000,0.996412,0.499101
3,aid_related,0.770309,0.758848,0.774537,0.762748
4,medical_help,0.800473,0.584990,0.928704,0.621434
5,medical_products,0.850129,0.592014,0.960301,0.639070
6,search_and_rescue,0.928420,0.531796,0.974306,0.553005
7,security,0.490394,0.500000,0.980787,0.495150
8,military,0.846699,0.546848,0.970139,0.576269
9,water,0.866521,0.726498,0.957292,0.777415


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000020EC6960280>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=LogisticRegression()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000020EC6960280>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=LogisticRegression()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [19]:
pipeline = sklearn.pipeline.Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
('tfidf', TfidfTransformer()),
('clf', SVC())])
parameters = {'clf__C': [0.5,1.0,1.5,2,2.5,3]}
 #'clf__estimator__penalty': ['l1', 'l2', 'elasticnet'],
 #'clf__estimator__solver': ['lbfgs','liblinear'],
 #'clf__estimator__verbose': [0,10,20]
 #}


grid_search = GridSearchCV(pipeline, param_grid=parameters, scoring='accuracy', cv=5)
model_parameters = []
# Fit the GridSearchCV object to the training data
for  i in range(y_test.shape[1]):
    grid_search.fit(X_train, y_train[:,i])
    best_params = grid_search.best_params_
    model_parameters.append(best_params)

KeyboardInterrupt: 

In [20]:
model_parameters

[{'clf__C': 1.5}, {'clf__C': 2}]

In [32]:
#best_params['clf__estimator__n_estimators']

In [25]:
pipeline.set_params(**best_params)
pipeline.fit(X_train, y_train)



c:\Users\avadi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000022FBAB533A0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.5)))])

In [26]:
y_pred = pipeline.predict(X_test)

Precision2 = []
Recall2 = []
F12 = []
Acc2 = []
for i in range(y_test.shape[1]):
    report = classification_report(y_test[:, i], y_pred[:, i], output_dict=True)
    
    macro_precision = report['macro avg']['precision']
    Precision2.append(macro_precision)
    
    macro_recall = report['macro avg']['recall']
    Recall2.append(macro_recall)
    
    macro_f1_score = report['macro avg']['f1-score']
    F12.append(macro_f1_score)

    accuracy = report['accuracy']
    Acc2.append(accuracy)

c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metr

In [27]:
Evaluate_2 = pd.DataFrame(
    {'Category': cats,
     'Precision' : Precision2,
     'Recall' : Recall2,
     'Accuracy': Acc2,
     'F1': F12
    })

Evaluate_2

,Category,Precision,Recall,Accuracy,F1
0,related,0.787128,0.698408,0.831250,0.725159
1,request,0.844214,0.761471,0.897685,0.793569
2,offer,0.498206,0.500000,0.996412,0.499101
3,aid_related,0.768800,0.759357,0.773843,0.762753
4,medical_help,0.794405,0.603091,0.929630,0.643663
5,medical_products,0.824957,0.604107,0.960069,0.652140
6,search_and_rescue,0.883299,0.540128,0.974421,0.566867
7,security,0.490394,0.500000,0.980787,0.495150
8,military,0.828791,0.566502,0.970602,0.605313
9,water,0.865844,0.742580,0.958565,0.789742


In [31]:
Evaluate_2.Accuracy.mean()

0.9489451058201062

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.